# welcome 


### Data description 
### Main dataset: 
### This dataset informs the medical situation of individuals segregation the observations by municipality, serializing them and providing income data of each one of them. Also, it provides an specific score about overall health & the case of an specific condition.

In [ ]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

url3 = r"Main_Dataset.csv"
md = pd.read_csv(url3)
url2 = r"Sociodemographics Data.csv"
sd = pd.read_csv(url2)
url1 = r"Municipality Pollution Data.csv"
mu = pd.read_csv(url1)


In [ ]:
# in case u want to always get the same sample use after the (n) after a coma random_state=: 
# This ensures that every time you run the code, you will get the same 500 rows sampled from your dataset (given the same input data).

md_AND_mu_merged = pd.merge(md, mu, on='Municipality')
md_r_sample = md_AND_mu_merged.sample(n=100,random_state=1)


fig = px.scatter_3d(md_r_sample, 
                     z='Overall_Health_Score', 
                     y='Traffic_Pollution', 
                     x='Air_Quality_Index',
                     color='Overall_Health_Score', 
                     opacity=0.65, 
                     size_max=20,
                     #hover_data=['Alcohol_Consumer','Smoker',"Physical_Activity",'Diet_Quality'],
                     #  color_discrete_sequence=px.colors.qualitative.Dark24
)

fig.update_layout(
    title="3D Scatter Plot of Health Score in relation with Traffic Pollution and AirQuality Index (Sampled Data)",
    scene=dict(
        yaxis_title="Pollution T.",
        zaxis_title="Health Score",
        xaxis_title=" AirQuality Index",
        bgcolor="black"  
    ),
    paper_bgcolor="black",  
    font=dict(color="white"),  
    scene_camera=dict(eye=dict(x=1.8, y=-1.8, z= 0.8))
)
fig.show()



### 1. **IMC > 30, Dieta poco saludable, No realiza actividad física**  
- Podría revelar una conexión clara con **diabetes tipo 2**, **síndrome metabólico** y **enfermedades articulares**.  
- **Idea:** Analizar si el sedentarismo y la dieta son los principales factores desencadenantes o si el IMC alto es el factor dominante.  



In [ ]:

md_r_sample = md.sample(n=10000,random_state=1).dropna(subset=["Income","Physical_Activity","BMI","Diet_Quality","Overall_Health_Score"])
fig = px.scatter(
    md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor')& Medical_Condition != 'Healthy'"), 
    y="BMI", 
    x="Overall_Health_Score", 
    color="Medical_Condition",
    hover_name="Medical_Condition",  
    color_discrete_sequence=px.colors.diverging.Spectral
)

fig.show()


In [ ]:

random_states = [10, 20, 30, 40, 50 ]  
fig = make_subplots(rows=1, cols=5, subplot_titles=[f"Random Set number: {rs}" for rs in random_states])

for i, rs in enumerate(random_states):
    md_r_sample = md.sample(n=10000, random_state=rs).dropna(subset=["Income", "Physical_Activity", "BMI", "Diet_Quality", "Overall_Health_Score"])
    filtered_data = md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor') & (Medical_Condition == 'Diabetes' |  Medical_Condition == 'Hypertension')")

    scatter = px.scatter(
    filtered_data, y="BMI", x="Overall_Health_Score",
    color="Medical_Condition",
    hover_name="Medical_Condition",
    size_max=10,  
    color_discrete_sequence=px.colors.diverging.Spectral
    )

    for trace in scatter.data:
        fig.add_trace(trace, row=1, col=i + 1)
fig.update_layout(title="Posible risk agents that present diabetes and Hypertension over different Random observations", 
    showlegend=False)


fig.show()



### 2. **Consumo excesivo de alcohol, Fumador, IMC > 25**  
- La combinación de tabaquismo y alcohol podría aumentar el riesgo de **enfermedades hepáticas** y **cáncer**.  
- **Idea:** Examinar si el consumo excesivo de alcohol o el tabaquismo tiene un impacto mayor en el desarrollo de cáncer.  

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=md[(md["Smoker"] == 1)]["BMI"],  
    histnorm='',
    name='Smoker',  
    marker_color='#440154',
    opacity=1,
    xbins=dict(
        start=25,  
        end=36,  
        size=1.5 
    )
))
fig.add_trace(go.Histogram(
    x=md[(md["Alcohol_Consumer"] == 1)]["BMI"],  
    histnorm='',
    name='Alcohol Consumer',
    marker_color='#22A884',
    opacity=1,
    xbins=dict(
        start=25,  
        end=35.5,  
        size=1.5  
    )
))

fig.update_layout(
    title_text='BMI Distribution of Cancer Risk Agents: Smokers vs Alcohol  ',
    xaxis_title_text='BMI',
    yaxis_title_text='Count',
    bargap=0.2,  
    bargroupgap=0.1,  
    barmode='group'  
)

fig.update_xaxes(range=[25, 35.5])
fig.show()


#### medias a traves de diferentes municipalidades 

In [ ]:

pollution_medians = mu.groupby("Municipality")["Traffic_Pollution"].median()

top_10_max = pollution_medians.nlargest(10)
top_10_min = pollution_medians.nsmallest(10)

mu_top_max = mu[mu["Municipality"].isin(top_10_max.index)]
mu_top_min = mu[mu["Municipality"].isin(top_10_min.index)]

fig_max = px.box(mu_top_max, x="Municipality",
                y="Traffic_Pollution",
                title="Top 10 Municipalities with Highest Traffic Pollution",
                category_orders={"Municipality": top_10_max.index})

fig_min = px.box(mu_top_min, x="Municipality",
                y="Traffic_Pollution",
                title="Top 10 Municipalities with Lowest Traffic Pollution",
                category_orders={"Municipality": top_10_min.index})


fig_max.update_traces(marker=dict(color="#440154")) 
fig_min.update_traces(marker=dict(color="#22A884"))


fig = make_subplots(rows=1, cols=2, subplot_titles=[
    "Top 10 Municipalities with Highest Traffic Pollution",
    "Top 10 Municipalities with Lowest Traffic Pollution"
])

for maxm in fig_max.data:
    fig.add_trace(maxm, row=1, col=1)

for minm in fig_min.data:
    fig.add_trace(minm, row=1, col=2)

fig.update_layout(
    width=1200, height=500,
    showlegend=False,
    font=dict(size=12)
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots


md_mu_merged = pd.merge(md, mu, on="Municipality")


income_medians = md_mu_merged.groupby("Municipality")["Income"].median()


top_10_max = income_medians.nlargest(10)
top_10_min = income_medians.nsmallest(10)


top_max_df = md_mu_merged.loc[md_mu_merged["Municipality"].isin(top_10_max.index)]
top_min_df = md_mu_merged.loc[md_mu_merged["Municipality"].isin(top_10_min.index)]

fig_max = px.box(top_max_df, x="Municipality", y="Income",
                 title="Top 10 Municipalities with Highest Income",
                 category_orders={"Municipality": top_10_max.index})

fig_min = px.box(top_min_df, x="Municipality", y="Income",
                 title="Top 10 Municipalities with Lowest Income",
                 category_orders={"Municipality": top_10_min.index})


fig_max.update_traces(marker=dict(color="#52d053")) 
fig_min.update_traces(marker=dict(color="#d3290f"))  


fig = make_subplots(rows=1, cols=2, subplot_titles=[
    "Top 10 Municipalities with Highest Income",
    "Top 10 Municipalities with Lowest Income"
])


for trace in fig_max.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig_min.data:
    fig.add_trace(trace, row=1, col=2)


fig.update_layout(
    width=1200, height=500,
    showlegend=False,
    font=dict(size=12),
    xaxis_tickangle=-45  
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots


md_mu_merged = pd.merge(md, mu, on="Municipality")


income_medians = md_mu_merged.groupby("Municipality")["Income"].median()


top_10_max = income_medians.nlargest(10)
top_10_min = income_medians.nsmallest(10)


top_max_df = md_mu_merged.loc[md_mu_merged["Municipality"].isin(top_10_max.index)]
top_min_df = md_mu_merged.loc[md_mu_merged["Municipality"].isin(top_10_min.index)]

fig_max = px.box(top_max_df, x="Municipality", y="Income",
                 title="Top 10 Municipalities with Highest Income",
                 category_orders={"Municipality": top_10_max.index})

fig_min = px.box(top_min_df, x="Municipality", y="Income",
                 title="Top 10 Municipalities with Lowest Income",
                 category_orders={"Municipality": top_10_min.index})


fig_max.update_traces(marker=dict(color="#52d053")) 
fig_min.update_traces(marker=dict(color="#d3290f"))  


fig = make_subplots(rows=1, cols=2, subplot_titles=[
    "Top 10 Municipalities with Highest Income",
    "Top 10 Municipalities with Lowest Income"
])


for trace in fig_max.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig_min.data:
    fig.add_trace(trace, row=1, col=2)


fig.update_layout(
    width=1200, height=500,
    showlegend=False,
    font=dict(size=12),
    xaxis_tickangle=-45  
)

fig.show()